# Compare Datasets

In [ ]:
import pandas as pd
import numpy as np
import pyreadstat
from reed import regex_select, compute_confusion
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500

%load_ext autoreload
%autoreload 2

## Load data & compare overlap of instances in datasets

In [ ]:
idc = 'xwaveid' # merge id
finn = pd.read_csv("data/all_vars.csv",index_col='xwaveid').reset_index()
anna, meta = pyreadstat.read_dta("../reduregvars.dta")
raw, meta_r = pyreadstat.read_sav(f'data/part1/Combined a190c.sav') 
raw['xwaveid'] = raw['xwaveid'].astype(int)
anna['xwaveid'] = anna['xwaveid'].astype(int)
finn['in_finn'] = True
anna['in_anna'] = True

### Check currently studying indicators

In [ ]:
summary_study = ['aedqstdy','aedfts','acaeft','acaept','anlreast','abncsty','abnfsty']

c11_study = [
 'aedcqsl',
 'aedcqsh',
 'aedcqnq',
 'aedcqtq',
 'aedcqta',
 'aedcqtc',
 'aedcqc1',
 'aedcqc2',
 'aedcqc3',
 'aedcqc4',
 'aedcqcd',
 'aedcqad',
 'aedcqav',
 'aedcqbd',
 'aedcqhd',
 'aedcqgd',
 'aedcqms',
 'aedcqdc',
 'aedcqbc',
 'aedcqsc',
 'aedcqcc',
 'aedcqgc',
 'aedcqcn',
 'aedcqdn',
 'aedcqnei',
 'aedcqna',
 'aedcqos',
 'aedcqdk',
]

dv_asced_study = [
 'aedcq100',
 'aedcq110',
 'aedcq120',
 'aedcq200',
 'aedcq211',
 'aedcq221',
 'aedcq310',
 'aedcq311',
 'aedcq312',
 'aedcq400',
 'aedcq411',
 'aedcq413',
 'aedcq421',
 'aedcq500',
 'aedcq511',
 'aedcq514',
 'aedcq521',
 'aedcq524',
 'aedcq600',
 'aedcq611',
 'aedcqunk'
]

studying_all = summary_study + c11_study + dv_asced_study

In [ ]:
for c in studying_all:
    if raw[c].max() > 1:
        print(c)
        print(raw[c].value_counts(dropna=False))
    

In [ ]:
def print_columns_with_labels(cols, meta, name):
    print(name)
    print('---------------------')
    for c in cols:
        print(c,"->",meta.column_names_to_labels.get(c))
    print('')
    
print_columns_with_labels(summary_study, meta_r, 'Summary columns')
print_columns_with_labels(c11_study, meta_r, 'C11 Questions')
print_columns_with_labels(dv_asced_study, meta_r, 'DV study')

In [ ]:

anna_ = pd.merge(anna, raw[studying_all+[idc]], how='left',on=idc)
print('aedcqfpt = ',meta_r.column_names_to_labels['aedcqfpt'])
anna_[studying_all].sum(axis=0)

### Agreement in treatment, outcome & instances

In [ ]:
# look at agreement of treatment, outcome and instances
treatments = ['^reduhl$', '^rehllt$', '^redudl$', '^redufl$', '^redllt$', '^refllt$']
outcomes = ['^rlwage$', '^mh$', '^mhbm$', '^wkhr$', '^y_']
compare = ['xwaveid',"^in_"]+treatments+outcomes+['ahgage1']

finn = finn[regex_select(finn.columns, compare)]
anna = anna[regex_select(anna.columns, compare)]
c = pd.merge(anna,finn,on=['xwaveid'],how='outer',suffixes=('_a','_f'))
c['in_finn'].fillna(False,inplace=True)
c['in_anna'].fillna(False,inplace=True)
both = c[c['in_anna']&c['in_finn']]
finn_only = c[~c['in_anna']&c['in_finn']]
anna_only = c[c['in_anna']&~c['in_finn']]
print(f'In both:{len(both)}, Finn only:{len(finn_only)}, Anna only:{len(anna_only)}')
assert len(both)+len(finn_only)+len(anna_only)==len(c), "merged frames must be same length"

## For people in both datasets
### Compare treatment computations

In [ ]:
treatment_pairs = [
    ('reduhl_a','reduhl_f'),
    ('redudl_a','redudl_f'),
    ('redufl_a','redufl_f')
]
for ta, tf in treatment_pairs:
    print("Comparing:",(ta, tf))
    confusion = compute_confusion(both[ta],both[tf],'anna','finn')
    display(confusion)
    assert confusion.iloc[1,0] < 20, 'large number of instances treated according to anna but not finn'

In [ ]:
both[both['redudl_a']!=both['redudl_f']].to_csv("data/anna_compare.csv",index=False)

### Compare outcome computations
   - These are almost identical. The difference is probably as I haven't replaced the values for those claiming to be working more than 100 hours per week.

In [ ]:
outcome_pairs = [
    ('mh','y_ghmh'),
    ('wkhr','y_jbhruc')
]
for oa, of in outcome_pairs:
    print("Comparing:",(oa, of))
    display(np.corrcoef(both[oa].fillna(-1),both[of].fillna(-1)))